## Introduction

Automatic differentiation (AD) is a family of techniques for efficiently and accurately evaluating derivatives of numeric functions expressed as computer programs. Application of AD includes Newton’s method for solving nonlinear equations, real-parameter optimization, probabilistic inference, and backpropagation in neural networks. AD has been extremely popular because of the booming development in machine learning and deep learning techniques. Our AD sofeware package enable user to calculate derivatives using the forward and reverse mode. 

**Mathematical Background**

Automatic Differentiation decomposes a complex function into a sequence of operations on elementary functions, evaluates the derivatives at each intermediate stage, repeatedly applies the chain rule to obtain the derivative of the outermost function.
We provides explanations for related math concepts below. 

**Elimentary functions**

The class of functions consisting of the polynomials, the exponential functions, the logarithmic functions, the trigonometric functions, the inverse trigonometric functions,and the functions obtained from those listed by the four arithmetic operations and by superposition(i.e. composition),applied by finitely many times.

**Chain Rule**
+ Used to compute the derivative of a composite function
+ Core of automatic differentiation
$$ f \circ g (x) = f(g(x))$$
$$\frac{d}{dx}[f(g(x))] = f'(g(x))g'(x)$$

**Dual Numbers**
+ Used to compute derivative for elementary functions in automatic differentiation
+ Replace x and y with $x+x'\epsilon$ and $y+y'\epsilon$. x' and y' are real numbers,$\epsilon$ is an abstract number with the property: $\epsilon^2=0$
+ Carry out operations, the dual part gives us the derivative

**Topological Graph**
+ Each node represent a variable
+ Arrows indicate topological orders(order of operations) and operations themselves.


**Forward Mode Autodifferentiation**

Follow the topological order and store the values of each variable in the nodes.
visit each node in topological order. Let x denote our innermost function. For variable $u_i=g_i(v)$ we already know $\frac{dv}{dx}$, calculate $\frac{du_i}{dx}= \frac{du_i}{dv}\frac{dv}{dx}$


**Reverse Mode Autodifferentiation**

Has forward computation and backward computation

    **Forward Computation**
Follow the topological order and store the values of each variable in each nodes.
    
    
    **Backward Computation**
let y denote our final output variable and $u_j$, $v_j$ denote the intermediate variables
1. Initialize all partial derivative $\frac{dy}{du_j}$ to 0 and dy/dy = 1
2. visit each node in reverse topological order. For variable $u_i=g_i(v_1,...,v_n)$ we already know $\frac{dy}{du_i}$, increment $\frac{dy}{dv_j}$ by $\frac{dy}{du_i}\frac{du_i}{dv_j}$


## How to Use dyfs

In [ ]:
"""
This cell is to be deleted.

** notes on the package name
* dyfs is the contatenation of the name of our group members: 
* Danyun - Yan - Feiyu - Sara, and it sounds like 'diffs', but 
* since Danyun  is not in our group anymore, we may want to 
* change the name later
"""

### Variables

We start with a `Var` (variable). Let's call it `x`. 

In [ ]:
import dyfs.forward as fwd

In [ ]:
x = fwd.Variable()

### Univariate Functions

The core class in `dyfs` is `Expr` (expression), and we can build up `Expr` from `Var` and other `Expr`. All functions are represented as `Expr`. All `Expr`, including `Var` which is the most elementary `Expr`, implements the `eval_at` method, which evaluates and returns the value and (partial) derivatives of this `Expr` at a certain point on its domain.

Here we create a `Expr` that represents $f(x) = x^2 + \sin(x)$. There is no need to call the `Expr` constructor explicitly, because the `**` operator on `x` is overloaded and will return an `Expr`. The `Sin.expr` function also returns an `Expr` representing $\sin(x)$.

In [ ]:
from fwd.ele_func import Sin

In [ ]:
f = x**2 + Sin.expr(x)

We can then evaluates $f(x)$'s value and derivative on $x = 42$. 

In [ ]:
val, der = f.eval_at(var_dict={x: 42})

### Multivariate Functions

Similar operations can be carried out on multivariate functions.

Here we create an `Expr` that represents $g(x, y) = \exp(x+y)$. Here the `Exp.exp` function also returns an `Expr`.

In [ ]:
from fwd.ele_func import Exp

In [ ]:
g = Exp.expr(x+y)

We can then evaluates $g(x)$'s value and derivative at $x = 3, y = 4$. Here the returned `der` is a dictionary with entries of the partial derivatives of `x` and `y`.

In [ ]:
val, der = g.eval_at(var_dict={x: 3, y: 4})

### Vector Functions

Similar operations can be carried out on vector functions.

Here we create an `VFun` (vector function) that represents $h(\begin{bmatrix}x\\y\end{bmatrix}) = \begin{bmatrix}f(x)\\g(x, y)\end{bmatrix}$.

In [ ]:
h = fwd.VFun(fun_dict={f: (x), g: (x, y)}, var_list=[x, y])

We can then evaluates $h(x)$'s value and derivative at $x = 3, y = 4$. Here the returned `der` is the Jacobian of $h$ at $x = 3, y = 4$.

## Software Organization

### Directory Structure

The organization of `dyfs` is as follows. 
```
dyfs/
    __init__.py
    forward/
        __init__.py
        Var.py
        Expr.py
        VFun.py
        ele_func/
            Sin.py
            Exp.py
        ...
        tests/
            Var_tests.py
            Expr_tests.py
            VFun_tests.py
            ...
    backward/
        ...
    usecase/
        ...
```

### Modules

There are 3 modules in `dyfs`: `dyfs.forward` module contains classes and methods for forward mode auto differentiation. The directory named `ele_func` under `dyfs.forward` is the directory where all elementary function classes live. `dyfs.backward` contains classes and methods for backward mode auto differentiation. `dyfs.usecase` module contains some examples built on `dyfs.forward` and `dyfs.backward`.

### Test Automation

`TravisCI` and `Coveralls` are used for test automation. The test suites for each module is included in the `tests` directory whithin the moduel directory.

### Distribution

`dyfs` is distributed with `PyPI`.

## Implementation

### dyfs.forward.Var

`Var` represents a variable. It implements the `eval_at` method that returns the evaluation of this variable at a certain point. 'Evaluation of this variable' sure sounds odd. This is to make the `Var` class compatible with `Expr` class: they should share the same interface (see `dyfs.forward.Expr`).

In [5]:
class Var:
    
    def __init__(self):
        
    """
    Evaluates this variable on the point represented by val_dict
    Params: 
        val_dict: dictionary, contains the variables and their values
    Return:
        eval: Eval, the evaluation of Var
    """
    def eval_at(self, val_dict):

### dyfs.forward.Eval

`Eval` represents an evaluation. It implements `val` and `der` methods, which return the the value part of this evaluation and the derivative part of this evaluation respectively. The `der` method returns a dictionary whose keys are references to `Var`. The value of each key is the partial derivative with respect to the key.

In [ ]:
class Eval:
    
    def __init__(self, val, der):
    
    """
    Return the value part of this evaluation
    Return:
        the value
    """
    def val(self):
    
    """
    Return the derivative part of this evaluation
    Return:
        the derivative
    """
    def der(self):

### dyfs.forward.Expr

`VFun` represents an expression. It has 2 private fields: `_ele_func` and `_sub_expr`. `_ele_func` is the element function of this expression and `_sub_expr` is the sub-expressions of this function. For example: for `h = fwd.VFun(fun_dict={f: (x), g: (x, y)}, var_list=[x, y])`, its `_funcs` will be a list containing `f` and `g`.

`VFun` implements the `eval_at` method that returns the evaluation of this expression at a certain point.

In [3]:
class Expr:
    
    def __init__(self, ele_func, sub_expr,)
    
    """
    _ele_func: reference to a class, 
    the element function (e.g. Sin, Exp...) of this expression
    """
    _ele_func
    
    """
    _sub_expr: a list,
    the sub-expressions of this expression
    """
    _sub_expr
    
    """
    Evaluates this variable on the point represented by val_dict
    Params: 
        val_dict: dictionary, contains the variables and their values
    Return:
        eval: Eval, the evaluation of Var
        
    ** notes on implementation **
    * eval_at should first pass the val_dict to the eval_at method of
    * its _sub_expr. The returned Eval should then be passed to the 
    * the eval_at method of _ele_func to fully evaluate this expression. 
    """
    def eval_at(self, val_dict):

### dyfs.forward.VFun

`VFun` represents a vector function. It has a private field `_funcs`, which is the emapping rule for each entry of 
    the output vector. For example: for `g = fwd.exp(x+y)`, its `_ele_func` will be pointing to `fwd.exp_eval`, and its `_sub_expr` will be pointing to a `Expr` that represents $x+y$ (In turn, this `Expr`'s `_ele_func` will be pointing to `fwd.add_eval`, and its `_sub_expr` will be pointing to a list containing a `Var` representing $x$ and a `Var` representing $y$).

`Expr` implements the `eval_at` method that returns the evaluation of this expression at a certain point.

In [ ]:
class VFun:
    
    """
    The functions representing the mapping rule for each entry of 
    the output vector
    """
    _funcs
    
    """
    Construct a vector function.
    Params:
        fun_dict: a dictionary, 
                  key: Expr, the functions representing the mapping rule
                       for each entry of the output vector
                  val: Var, the variables involved in each function
        var_dict: a list, the Var objects involved in this VFun object
    """
    def __init__(self, fun_dict, var_list):
    
    """
    Evaluates this variable on the point represented by val_dict
    Params: 
        val_dict: dictionary, contains the variables and their values
    Return:
        eval: Eval, the evaluation of Var, whose derivative part should
            be a Jacobian
        
    ** notes on implementation **
    * Call the eval_at function of each Expr in _funcs
    """
    def eval_at(self, val_dict)

### dyfs.forward.ele_func.Sin

The class representing the elementary function $x \mapsto \sin(x)$. It implements 2 functions: `expr` and `eval_at` is a factory method that contruct an `Expr` of $\sin(x)$. `eval_at` is the method that does the calculation to evaluate $sin(x)$ at certain point.

In [ ]:
class Sin:
    
    def __init__(self):
    
    """
    Contruct and retur an Expr object
    
    ** notes on implementation **
    * this method should set the _sub_expr field of the constructed Expr
    * as well as the _ele_func field.
    """
    def expr(self, sub_expr):
    
    """
    Evaluate the elementary function of sin, using the evaluations of 
    sub-expressions.
    
    ** notes on implementation **
    * this method should set the _sub_expr field of the constructed Expr
    * as well as the _ele_func field.
    """
    def eval_at(self, evals):   

### dyfs.forward.ele_func.Exp

The class representing the elementary function $x \mapsto \exp(x)$. It implements 2 functions: `expr` and `eval_at` is a factory method that contruct an `Expr` of $\sin(x)$. `eval_at` is the method that does the calculation to evaluate $exp(x)$ at certain point.

In [ ]:
class Exp:
    
    def __init__(self):
    
    """
    Contruct and retur an Expr object
    
    ** notes on implementation **
    * this method should set the _sub_expr field of the constructed Expr
    * as well as the _ele_func field.
    """
    def expr(self, sub_expr):
    
    """
    Evaluate the elementary function of exp, using the evaluations of 
    sub-expressions.
    
    ** notes on implementation **
    * this method should set the _sub_expr field of the constructed Expr
    * as well as the _ele_func field.
    """
    def eval_at(self, evals):  